## Realizando a conexão com o MongoDB

Nesse momento vamos utilizar o código fornecido no MongoDB Atlas para realizarmos a conexão com o nosso banco de dados utilizando a biblioteca pymongo.

In [17]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
from dotenv import load_dotenv

load_dotenv()

uri = os.getenv('MONGO_URI')

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


## Criando o banco de dados e uma coleção

Conectados ao banco de dados, está tudo pronto para realizar operações nele. Primeiramente, vamos criar nossa base de dados chamada "db_produtos" e uma coleção dentro dela chamada "produtos". Vale lembrar que para um banco de dados do MongoDB ser efetivamente criado, ele precisa conter dados.

In [18]:
db = client["db_produtos"]
collection = db["produtos"]

In [19]:
client.list_database_names()

['db_produtos', 'admin', 'local']

In [20]:
# product = {"produto": "computador", "quantidade": 77}
# collection.insert_one(product)

InsertOneResult(ObjectId('668f2417ca21ee119a7b8cc8'), acknowledged=True)

In [21]:
collection.find_one()

{'_id': ObjectId('668f2265ca21ee119a7b8cc5'),
 'produto': 'computador',
 'quantidade': 77}

In [22]:
client.list_database_names()

['db_produtos', 'admin', 'local']

## Extraindo os dados da API

Agora que nossa base de dados e coleção estão criadas, podemos prosseguir para a etapa de extração dos dados que desejamos adicionar a esse banco de dados.

In [23]:
import requests

response = requests.get('https://labdados.com/produtos')
response.json()[0]

{'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'lat': -13.29,
 'lon': -41.71}

In [24]:
len(response.json())

9435

## Adicionando os dados extraídos na coleção

Com os dados extraídos no formato JSON, que é compatível com o MongoDB, podemos adicioná-los facilmente ao nosso banco de dados. Podemos utilizar o método `insert_many()` para adicionar todos os dados extraídos de uma vez.

In [25]:
docs = collection.insert_many(response.json())

In [26]:
len(docs.inserted_ids)

9435

In [27]:
collection.count_documents({})

9437

**Removendo o registro de exemplo**

In [30]:
remove_id = collection.find_one()["_id"]

In [31]:
# collection.delete_one({"_id": remove_id})

DeleteResult({'n': 1, 'electionId': ObjectId('7fffffff0000000000000017'), 'opTime': {'ts': Timestamp(1720657240, 5), 't': 23}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1720657240, 5), 'signature': {'hash': b'Vg\xf2x\xfaE;\xc7E\x10Rw\xb8\x8b\x9a=r\xd4\xbd_', 'keyId': 7340665846978576388}}, 'operationTime': Timestamp(1720657240, 5)}, acknowledged=True)

In [32]:
collection.find_one()

{'_id': ObjectId('668f2459ca21ee119a7b8cc9'),
 'Produto': 'Modelagem preditiva',
 'Categoria do Produto': 'livros',
 'Preço': 92.45,
 'Frete': 5.6096965236,
 'Data da Compra': '01/01/2020',
 'Vendedor': 'Thiago Silva',
 'Local da compra': 'BA',
 'Avaliação da compra': 1,
 'Tipo de pagamento': 'cartao_credito',
 'Quantidade de parcelas': 3,
 'lat': -13.29,
 'lon': -41.71}

In [33]:
client.close()